In [ ]:
from hcmus.utils import data_utils, transform_utils

splits = data_utils.get_data_splits()

In [ ]:
import random
from torchvision import transforms as T

def get_transforms():
    transform_train = T.Compose([
        T.Lambda(lambda img: T.Resize(random.randint(32, 224))(img)),
        T.Resize((224, 224)),
        T.RandomHorizontalFlip(p=0.5),
        T.RandomRotation(180),
        T.ColorJitter(
            brightness=0.2,
            contrast=0.0,
            saturation=0.0,
            hue=0.0
        ),
        T.RandomResizedCrop(
            size=224,
            scale=(0.8, 1.2),
            ratio=(0.75, 1.3333)
        ),
        T.RandAugment(num_ops=5),
        T.ToTensor()
    ])

    transform_test = T.Compose([
        # T.Resize((size, size)),
        T.Resize((224, 224)),
        T.ToTensor()
    ])

    return transform_train, transform_test

In [ ]:
transform_train, transform_test = get_transforms()
datasets = data_utils.get_image_datasets_v2(splits, transform_train, transform_test, random_margin=0.2)
dataloaders = data_utils.get_data_loaders_v2(datasets, {
    "train": True
})

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20, 20))
idx = 0
for x in range(5):
    for y in range(5):
        plt.subplot(5, 5, idx + 1)
        image, _, task = datasets["train"][idx]
        image = image.detach().numpy().transpose(1, 2, 0)
        plt.imshow(image)
        plt.title(task.get("task_id"))
        idx += 1

In [ ]:
from hcmus.models.backbone import CLIPBackbone
from hcmus.models.backbone import DinoBackbone
from hcmus.models.transfer import TransferNetwork
from hcmus.models.transfer import TransferTrainer

In [ ]:
backbone_list = [
    # (DinoBackbone, {"model_id": "facebook/dinov2-small"}),
    # (DinoBackbone, {"model_id": "facebook/dinov2-base"}),
    # (DinoBackbone, {"model_id": "facebook/dino-vitb8"}),
    # (DinoBackbone, {"model_id": "facebook/dino-vits8"}),
    # (DinoBackbone, {"model_id": "facebook/dino-vits16"}),
    # (DinoBackbone, {"model_id": "facebook/dino-vitb16"}),
    (CLIPBackbone, {"backbone_name": "ViT-B/32"}),
    (CLIPBackbone, {"backbone_name": "ViT-B/16"}),
]

In [ ]:
import mlflow
def get_or_create_experiment():
    name = "/TransferLearning"
    try:
        mlflow.create_experiment(name)
    except:
        pass

    return mlflow.get_experiment_by_name(name).experiment_id

In [11]:
from hcmus.utils import mlflow_utils
get_or_create_experiment()
for cls, params in backbone_list:
    mlflow.end_run()
    backbone = cls(**params)
    model = TransferNetwork(backbone, output_dim=99)
    trainer = TransferTrainer(
        model=model,
        mlflow_experiment_name="/TransferLearning",
        mlflow_run_name=cls.__name__ + f"/{params}"
    )
    trainer.train(64, dataloaders["train"], dataloaders["val"])
    trainer.load_model("best_model.pt")
    trainer.evaluate(dataloaders["test"], log_mlflow=True)
    mlflow_utils.log_json_artifact(datasets["train"].label2idx, "label2idx.json")
    trainer.end_run()

Epoch 6/64: 100%|██████████| 84/84 [01:03<00:00,  1.33it/s, Loss=1.4005]
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model saved to best_model.pt


2025/07/11 20:22:58 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'item-identification'}
2025/07/11 20:22:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 6/64:
  Train Loss: 1.6459
  Val Loss: 1.4321
  Val Accuracy: 0.6270
  Time: 178.07s
--------------------------------------------------


Epoch 7/64: 100%|██████████| 84/84 [01:01<00:00,  1.37it/s, Loss=1.7660]
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch 64/64:
  Train Loss: 0.5099
  Val Loss: 1.0325
  Val Accuracy: 0.7597
  Time: 271.54s
--------------------------------------------------


2025/07/12 03:48:25 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'item-identification'}
2025/07/12 03:48:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model loaded from best_model.pt


Evaluating: 100%|██████████| 187/187 [06:03<00:00,  1.95s/it]
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



EVALUATION RESULTS
Loss: 0.9530
Accuracy: 0.7593

Micro Averages:
  Precision: 0.7593
  Recall: 0.7593
  F1-Score: 0.7593

Macro Averages:
  Precision: 0.7417
  Recall: 0.7441
  F1-Score: 0.7035
🏃 View run CLIPBackbone/{'backbone_name': 'ViT-B/16'} at: http://jimica.ddns.net:5050/#/experiments/4/runs/8fbfaaa5eac140acacf7121bc7e9f1ab
🧪 View experiment at: http://jimica.ddns.net:5050/#/experiments/4
